In [ ]:
""" 
1. 环境
---
Google Colab GPU 进行下面方式安装，自己电脑可以按照自己方式进行安装；
YOLOX 安装 0.3.0 分支版本，不要使用仓库最新代码，可能存在未知无法解决的错误；
"""

%cd /content/
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX
# 切换到 0.3.0 版本
!git checkout -b 0.3.0 0.3.0
%pip install -U pip && pip install -r requirements.txt
%pip install -v -e . 


In [1]:
"""
2. 模型
---
使用 yolox_s 模型，进行迁移学习，可使用 wget 进行下载
"""
# %cd /content/drive/MyDrive/yolox-map-demo/
# !pip install wget

def download_models():
    """  
    下载 yolox_s 模型
    """
    import os
    import wget
    if not os.path.exists('models/yolox_s.pth'):
        os.mkdir('models')
        wget.download('https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth','models')
    print('models/yolox_s.pth')


download_models()

models/yolox_s.pth


In [3]:
"""
3. 数据集
---
使用 labelimg 进行图片标注，将标注文件和图片放到一个文件夹中
下载 Google Drive Datasets 放到该文件夹，并解压
数据集: https://drive.google.com/file/d/12n1GVrUX4yJHUCDOx-4CPoA2ooiSU5L2/view?usp=share_link
"""

def datasets_download():
    """
    下载数据集 
    """
    import os 
    if not  os.path.exists('datasets'):
        os.mkdir('datasets')
    # 若使用 colab ，可直接挂载 drive ，进行复制操作；否则自行下载，并进行解压
    if not os.path.exists('datasets/crack-datasets'):
        os.system('cp /content/drive/MyDrive/my-datasets/crack-datasets.zip ./datasets/')

    
# 下载
datasets_download()

In [6]:
"""
4. 数据集转换 
---
将 labelimg 标注数据集，转换为 coco 数据集，使用 tools/labelimg2coco.py 
"""

def datasets_init_coco():
    from tools import labelimg2coco
    labelimg2coco(
        labelimg_dir='datasets/crack-datasets',
        coco_dir='datasets/crack',
        classes=['0'],
        image_suffix='.jpg'
    )

datasets_init_coco()

datasets/crack/annotations/instances_train2017.json
datasets/crack/annotations/instances_val2017.json
datasets/crack/annotations/instances_test2017.json
datasets/crack/train.txt
datasets/crack/val.txt
datasets/crack/test.txt
datasets/crack/class.txt
train size 28 , val size 8 , test size 4


In [ ]:
"""
5. 训练 
---
复制 YOLOX/exps/example/custom/yolox_s.py 文件，并进行变更内容，见 crack_exp_yolox_s.py 
"""

# %cd /content/drive/MyDrive/yolox-crack-demo/
!python train.py -f crack_exp_yolox_s.py -d 1 -b 16 --fp16 -o -c models/yolox_s.pth

In [ ]:
"""
6. 测试 
---
测试图片在 asserts/demo.jpg , 使用训练完毕的模型进行测试
训练完毕的模型地址：/YOLOX_outputs/best_ckpt.pth
需要修改 demo.py 中的 COCO_CLASS 内容
"""
# %cd /content/drive/MyDrive/yolox-crack-demo/
# 修改 demo.p/COCO_CLASS 内容
# 复制最新模型到 models 文件夹
%cp ./YOLOX_outputs/crack_exp_yolox_s/best_ckpt.pth ./models/
# 测试
!python demo.py image -f crack_exp_yolox_s.py -c models/best_ckpt.pth --path assets/demo.png --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

In [ ]:
""" 
7. 评估模型
"""

!python eval.py -f crack_exp_yolox_s.py -c models/best_ckpt.pth -b 16 -d 1 --conf 0.001 --fp16 --fuse